---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [14]:
def graph_identification():
    
    # Your Code Here
    algorithms = []
    
    for G in P1_Graphs:
        clustering = nx.average_clustering(G)
        
        degrees = G.degree()
        degree_values = sorted(set(degrees.values()))
        degree_hist = [list(degrees.values()).count(i) / float(nx.number_of_nodes(G)) for i in degree_values]
        
        if len(degree_hist) > 10:
            algorithms.append('PA')
        elif clustering < 0.1:
            algorithms.append('SW_H')
        else:
            algorithms.append('SW_L')

    
    return algorithms # Your Answer Here

graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [15]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [23]:
def salary_predictions():
    
    # Your Code Here
    from sklearn.neural_network import MLPClassifier
    from sklearn.preprocessing import MinMaxScaler
    
    # Initialize the dataframe, using the nodes as the index
    df = pd.DataFrame(index=G.nodes())
    
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Departmnt'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    
    
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['closeness'] = pd.Series(nx.closeness_centrality(G, normalized = True))
    df['betweeness'] = pd.Series(nx.betweenness_centrality(G, normalized = True))
    df['page_rank'] = pd.Series(nx.pagerank(G))
    
    # Create Training and Test sets
    df_train = df[~pd.isnull(df['ManagementSalary'])]
    df_test = df[pd.isnull(df['ManagementSalary'])]
    
    features = ['clustering', 'degree', 'degree_centrality', 'closeness', 'betweeness', 'page_rank']
    
    X_train = df_train[features]
    y_train = df_train['ManagementSalary']
    X_test = df_test[features]
    
    # Scale the Training and Test sets
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Use a Multi-layer Perceptron classifier.
    clf = MLPClassifier(hidden_layer_sizes = [7, 3], alpha = 5, random_state = 0, solver='lbfgs', verbose=0)
    
    # Fit the results    
    clf.fit(X_train_scaled, y_train)    
                        
    # Predict the probability
    test_probability = clf.predict_proba(X_test_scaled)[:, 1]
                        
    # Tranform the results into the desired state
    predictions = pd.Series(test_probability, X_test.index)
    
    
    return predictions # Your Answer Here

salary_predictions()

1       0.128546
2       0.585968
5       0.979772
8       0.133637
14      0.304816
18      0.208538
27      0.269533
30      0.335903
31      0.169229
34      0.143584
37      0.096389
40      0.195267
45      0.162253
54      0.249830
55      0.252461
60      0.177178
62      0.993639
65      0.550807
77      0.078467
79      0.131995
97      0.099871
101     0.057601
103     0.214167
108     0.122891
113     0.454294
122     0.044497
141     0.396262
142     0.863655
144     0.072048
145     0.419359
          ...   
913     0.045870
914     0.078632
915     0.002114
918     0.144297
923     0.035376
926     0.090159
931     0.056074
934     0.014007
939     0.011224
944     0.002380
945     0.039772
947     0.162779
950     0.098119
951     0.038487
953     0.027446
959     0.002951
962     0.002469
963     0.139814
968     0.110503
969     0.104857
974     0.095090
984     0.012740
987     0.110421
989     0.113566
991     0.117109
992     0.003989
994     0.003107
996     0.0024

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [24]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [25]:
def new_connections_predictions():
    
    # Your Code Here
    from sklearn.neural_network import MLPClassifier
    from sklearn.preprocessing import MinMaxScaler
        
    # Compute the preferential attachment score of all node pairs
    preferential_attachment = list(nx.preferential_attachment(G))
    
    df = pd.DataFrame(index=[(x[0], x[1]) for x in preferential_attachment])
    df['preferential_attachment'] = [x[2] for x in preferential_attachment]
    
    # Return iterator of 3-tuples in the form (u, v, p) where (u, v) is a pair of nodes and p is their score.
    cn_soundarajan_hopcroft = list(nx.cn_soundarajan_hopcroft(G, community='Department'))
    
    # Clean and transform the data
    df_cn_soundarajan_hopcroft = pd.DataFrame(index=[(x[0], x[1]) for x in cn_soundarajan_hopcroft])
    df_cn_soundarajan_hopcroft['cn_soundarajan_hopcroft'] = [x[2] for x in cn_soundarajan_hopcroft]
    df = df.join(df_cn_soundarajan_hopcroft, how='outer')
    df['cn_soundarajan_hopcroft'] = df['cn_soundarajan_hopcroft'].fillna(value=0)
    
    # Add features into current data frame
    df['resource_allocation_index'] = [x[2] for x in list(nx.resource_allocation_index(G))]
    df['jaccard_coefficient'] = [x[2] for x in list(nx.jaccard_coefficient(G))]
    df = future_connections.join(df,how='outer')

    # Create Train and Test data frames
    df_train = df[~pd.isnull(df['Future Connection'])]
    df_test = df[pd.isnull(df['Future Connection'])]
    
    # Create Training and Testing sets 
    features = ['cn_soundarajan_hopcroft', 'preferential_attachment', 'resource_allocation_index', 'jaccard_coefficient']
    
    X_train = df_train[features]
    y_train = df_train['Future Connection']
    X_test = df_test[features]
    
    # Scale the Training and Test sets
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Use a Multi-layer Perceptron classifier.
    clf = MLPClassifier(hidden_layer_sizes = [7, 3], alpha = 5, random_state = 0, solver='lbfgs', verbose=0)
    
    # Fit the results    
    clf.fit(X_train_scaled, y_train)    
                        
    # Predict the probability
    test_probability = clf.predict_proba(X_test_scaled)[:, 1]
    
    # Tranform the results into the desired state
    predictions = pd.Series(test_probability, X_test.index)
    target = future_connections[pd.isnull(future_connections['Future Connection'])]
    target['probability'] = [predictions[x] for x in target.index]
    
    return target['probability'] # Your Answer Here

new_connections_predictions()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(107, 348)    0.029429
(542, 751)    0.011141
(20, 426)     0.610907
(50, 989)     0.011278
(942, 986)    0.011427
(324, 857)    0.011259
(13, 710)     0.157015
(19, 271)     0.115063
(319, 878)    0.011326
(659, 707)    0.011155
(49, 843)     0.011369
(208, 893)    0.011188
(377, 469)    0.006547
(405, 999)    0.020066
(129, 740)    0.017427
(292, 618)    0.020915
(239, 689)    0.011335
(359, 373)    0.008223
(53, 523)     0.091909
(276, 984)    0.011362
(202, 997)    0.011390
(604, 619)    0.066696
(270, 911)    0.011333
(261, 481)    0.071797
(200, 450)    0.995665
(213, 634)    0.011099
(644, 735)    0.053715
(346, 553)    0.010762
(521, 738)    0.010132
(422, 953)    0.018430
                ...   
(672, 848)    0.011333
(28, 127)     0.959157
(202, 661)    0.010706
(54, 195)     0.999939
(295, 864)    0.011238
(814, 936)    0.011120
(839, 874)    0.011427
(139, 843)    0.011250
(461, 544)    0.009705
(68, 487)     0.009631
(622, 932)    0.011205
(504, 936)    0.016295
(479, 528) 